In [1]:
import numpy as np
import  pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from plot_keras_history import plot_history
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
data=pd.read_csv('DATA.csv')

In [3]:
data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [4]:
data['label']=data['label'].replace(2,1)

In [5]:
X=data['combined_result']
y=data['label']
time_series=data['t_dist']

In [6]:
std_X = X.std()
desired_snr = 0.1
std_noise = std_X * desired_snr
noise = np.random.normal(0, std_noise, X.shape)

In [7]:
data['noisy_feature_column'] = X + noise
noisy_signal=data['noisy_feature_column']

In [8]:
X.shape

(299995,)

In [9]:
X=pd.concat([X, noisy_signal], ignore_index=True)
y=pd.concat([y, y], ignore_index=True)

In [10]:
X.shape

(599990,)

In [11]:
# Extracting sequences from the dataset
sequences=[]
labels=[]
temp=[]

for index,value in y.items():
    if value==1:
        temp.append(X[index])
        if (index+1)<len(y) and y[index+1]==0:
            sequences.append(temp)
            labels.append(1)
            temp=[]
    if value==0:
        temp.append(X[index])
        if (index+1)<len(y) and y[index+1]==1:
            sequences.append(temp)
            labels.append(0)
            temp=[]

In [12]:
# Not a sequence but label is 1 
zero_seq_label_one=[]
for position, sublist in enumerate(sequences):
    if all(element == 0 for element in sublist):
        if labels[position]==1:
            zero_seq_label_one.append(position)

In [13]:
print(len(zero_seq_label_one))

58


In [14]:
# Replacing those with 0
for value in zero_seq_label_one:
    labels[value]=0

We are replacing labels of non sequences to zero because it is possible that due to combine data we still have the previous data problem.
But we are not correcting the 2nd scenerio where is a sequence but label 0 because after noise addition there are some sequences that were not sequence in orignal form and after noise they are still not sequence so if we replace labels for those sequences they also get changed. It is also clear from number of those sequences previously in orignal data they were 127  here they are 800 something. 

In [15]:
# Padding and reshaping
X_train=pad_sequences(sequences,padding='post',truncating='post',dtype=float,maxlen=50)

In [16]:
y_train = np.array(labels)
y_train = y_train.reshape(-1, 1)

In [17]:
y_train.shape

(2899, 1)

In [18]:
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(2899, 50, 1)

In [19]:
# It is because Our each  sequence has max laength of 50 and in output we want to predict a label
Tx=50
Ty=1
repeator = tf.keras.layers.RepeatVector(Tx)
concatenator = tf.keras.layers.Concatenate(axis=-1)
densor1 =tf.keras.layers. Dense(10, activation = "tanh")
densor2 = tf.keras.layers.Dense(1, activation = "relu")
activator = tf.keras.layers.Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor =tf.keras.layers. Dot(axes = 1)

In [20]:
def one_step_attention(a,s_prev):
    # We done this to change s_prev to shape of(m,Tx,n_s) for cocatination with a, because concatenation requires all dimension same accept concat axis in this 
    # case is last one so a and s_prev have same first two dims (m,Tx) but last one is changed
    s_prev=repeator(s_prev)
    # We will here concatenate a and s_prev
    concat=concatenator([a,s_prev])
    # here i will calculate energies with 2 dense layers
    e=densor1(concat)
    energies=densor2(e)
    # we know alpha is softmax of this energy
    alpha=activator(energies)
    # to calculate context vector we take dot product of alpha and a
    context_vector=dotor([alpha,a])
    return context_vector

In [21]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.
post_activation_LSTM_cell = tf.keras.layers.LSTM(n_s, return_state = True) # Please do not modify this global variable.
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [22]:
def modelf(Tx,Ty,n_a, n_s):
    X=tf.keras.layers.Input(shape=(X_train.shape[1],1)) # because embedding layer only demands the sequence length if i give full shape like (m,Tx) the
    # ouput of embedding layer will be 4D which can not be fed into BILSTM
    # hidden state for post LSTM
    s0 = tf.keras.layers.Input(shape=(n_s,), name='s0')
    # cell state for post lstm
    # because we know From CampusX that shape of hidden and cell state of lstm are equal
    c0 = tf.keras.layers.Input(shape=(n_s,), name='c0')
    s=s0
    c=c0
    outputs = []

    a = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_a,return_sequences=True),input_shape=(X_train.shape[1],1))(X)

    for t in range(Ty):
        context=one_step_attention(a,s)
        _,s,c=post_activation_LSTM_cell(context,initial_state = [s,c] )
        out = output_layer(s)
        outputs.append(out)



    print(outputs)
    model=tf.keras.models.Model(inputs=[X,s0,c0],outputs=outputs)

    return model

In [23]:
model = modelf(Tx, Ty, n_a, n_s)

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_2')>]


In [24]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00001,beta_1=0.9,beta_2=0.999)#0.00001
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = opt, metrics = ['accuracy'])

In [25]:
# For Training Set
m=X_train.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

In [33]:
model_history=model.fit([X_train, s0, c0], y_train, epochs=200, batch_size=100)#350

Epoch 1/200
29/29 [==============================] - 5s 22ms/step - loss: 0.6936 - accuracy: 0.5412
Epoch 2/200
29/29 [==============================] - 1s 22ms/step - loss: 0.6903 - accuracy: 0.5847
Epoch 3/200
29/29 [==============================] - 1s 22ms/step - loss: 0.6871 - accuracy: 0.6199
Epoch 4/200
29/29 [==============================] - 1s 22ms/step - loss: 0.6840 - accuracy: 0.6371
Epoch 5/200
29/29 [==============================] - 1s 22ms/step - loss: 0.6811 - accuracy: 0.6444
Epoch 6/200
29/29 [==============================] - 1s 22ms/step - loss: 0.6781 - accuracy: 0.6475
Epoch 7/200
29/29 [==============================] - 1s 23ms/step - loss: 0.6753 - accuracy: 0.6513
Epoch 8/200
29/29 [==============================] - 1s 22ms/step - loss: 0.6725 - accuracy: 0.6516
Epoch 9/200
29/29 [==============================] - 1s 23ms/step - loss: 0.6697 - accuracy: 0.6530
Epoch 10/200
29/29 [==============================] - 1s 23ms/step - loss: 0.6670 - accuracy: 0.6561

In [34]:
plot_history(model_history,show_standard_deviation=False,show_average=True)

<IPython.core.display.Javascript object>

(<Figure size 1000x500 with 2 Axes>,
 array([<Axes: title={'center': 'Loss'}, xlabel='Epochs', ylabel='Loss'>,
        <Axes: title={'center': 'Accuracy'}, xlabel='Epochs', ylabel='Accuracy'>],
       dtype=object))

In [26]:
#model.save_weights('Model-Trained-On-Noise+Normal.h5')
model.load_weights('Model-Trained-On-Noise+Normal.h5')

**Testing On Clean Data**

In [27]:
min_10_data=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')

In [28]:
min_10_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
min_10_data.shape

(59999, 3)

In [29]:
min_10_data['label']=min_10_data['label'].replace(2,1)

In [30]:
x_values=min_10_data['combined_result']
y_values=min_10_data['label']

In [31]:
event=[]
labe=[]
temp=[]
for index,value in y_values.items():
    if value==1:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==0:
            event.append(temp)
            labe.append(1)
            temp=[]
    if value==0:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==1:
            event.append(temp)
            labe.append(0)
            temp=[]

In [32]:
# Not a sequence but label is 1 
zero_seq_label_one=[]
for position, sublist in enumerate(event):
    if all(element == 0 for element in sublist):
        if labe[position]==1:
            zero_seq_label_one.append(position)

In [33]:
# Replacing those with 0
for value in zero_seq_label_one:
    labe[value]=0

In [34]:
# Is a sequence but label 0
seq_label_zero=[]
for position, sublist in enumerate(event):
    if any(element != 0 for element in sublist):
        if labe[position]==0:
            seq_label_zero.append(position)

In [35]:
# Replacing those with 1
for value in seq_label_zero:
    labe[value]=1

In [36]:
# Checking the Number of events
zero=0
one=0
for value in labe:
    if value==0:
        zero+=1
    if value==1:
        one+=1
print(zero)
print(one)

140
149


In [38]:
event=pad_sequences(event,padding='post',truncating='post',dtype=float,maxlen=50)
labe=np.array(labe)
labe=labe.reshape(-1,1)
event=event.reshape(event.shape[0], event.shape[1], 1)

In [39]:
# For 10 Min Data set
m_10=event.shape[0]
s0_10 = np.zeros((m_10, n_s))
c0_10 = np.zeros((m_10, n_s))

In [40]:
predictions_10=model.predict([event, s0_10, c0_10])
predictions_10=np.round(predictions_10)

10/10 [==============================] - 2s 6ms/step


In [41]:
from sklearn.metrics import confusion_matrix


# Get the confusion matrix
cm = confusion_matrix(labe, predictions_10)

# We will store the results in a dictionary for easy access later
per_class_accuracies = {}
classes=[0,1]
# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(classes):
    # True negatives are all the samples that are not our current GT class (not the current row) 
    # and were not predicted as the current class (not the current column)
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))

    # True positives are all the samples of our current GT class that were predicted as such
    true_positives = cm[idx, idx]

    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm)

In [42]:
# Correct Accuracy
print(per_class_accuracies)

{0: 0.8408304498269896, 1: 0.8408304498269896}


In [103]:
report = classification_report(predictions_10,labe)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      0.75      0.86       186
         1.0       0.69      1.00      0.82       103

    accuracy                           0.84       289
   macro avg       0.85      0.88      0.84       289
weighted avg       0.89      0.84      0.84       289


**Testing On Combined Data**

In [43]:
min_10_data=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')

In [44]:
min_10_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
min_10_data.shape

(59999, 3)

In [45]:
min_10_data['label']=min_10_data['label'].replace(2,1)

In [46]:
x_values=min_10_data['combined_result']
y_values=min_10_data['label']

In [47]:
std_X = x_values.std()
desired_snr = 0.1
std_noise = std_X * desired_snr
noise = np.random.normal(0, std_noise, x_values.shape)

In [48]:
min_10_data['noisy_feature_column'] = x_values + noise
noisy_signal=min_10_data['noisy_feature_column']

In [49]:
x_values=pd.concat([x_values, noisy_signal], ignore_index=True)
y_values=pd.concat([y_values, y_values], ignore_index=True)

In [50]:
event=[]
labe=[]
temp=[]
for index,value in y_values.items():
    if value==1:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==0:
            event.append(temp)
            labe.append(1)
            temp=[]
    if value==0:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==1:
            event.append(temp)
            labe.append(0)
            temp=[]

In [51]:
# Not a sequence but label is 1 
zero_seq_label_one=[]
for position, sublist in enumerate(event):
    if all(element == 0 for element in sublist):
        if labe[position]==1:
            zero_seq_label_one.append(position)

In [52]:
# Replacing those with 0
for value in zero_seq_label_one:
    labe[value]=0

In [53]:
# Checking the Number of events
zero=0
one=0
for value in labe:
    if value==0:
        zero+=1
    if value==1:
        one+=1
print(zero)
print(one)

301
278


In [54]:
event=pad_sequences(event,padding='post',truncating='post',dtype=float,maxlen=50)
labe=np.array(labe)
labe=labe.reshape(-1,1)
event=event.reshape(event.shape[0], event.shape[1], 1)

In [55]:
# For 10 Min Data set
m_10=event.shape[0]
s0_10 = np.zeros((m_10, n_s))
c0_10 = np.zeros((m_10, n_s))

In [56]:
predictions_10 = model.predict([event, s0_10, c0_10])
predictions_10 = np.round(predictions_10)

19/19 [==============================] - 0s 8ms/step


In [57]:
# Get the confusion matrix
cm = confusion_matrix(labe, predictions_10)

# We will store the results in a dictionary for easy access later
per_class_accuracies = {}
classes=[0,1]
# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(classes):
    # True negatives are all the samples that are not our current GT class (not the current row) 
    # and were not predicted as the current class (not the current column)
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))

    # True positives are all the samples of our current GT class that were predicted as such
    true_positives = cm[idx, idx]

    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm)

In [58]:
# Correct Accuracy
print(per_class_accuracies)

{0: 0.8169257340241797, 1: 0.8169257340241797}


In [119]:
report = classification_report(predictions_10,labe)
print(report)

              precision    recall  f1-score   support

         0.0       0.94      0.77      0.85       370
         1.0       0.69      0.92      0.79       209

    accuracy                           0.82       579
   macro avg       0.82      0.84      0.82       579
weighted avg       0.85      0.82      0.83       579


**Testing On Noisy Data Only**

In [59]:
min_10_data=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')

In [60]:
min_10_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
min_10_data.shape

(59999, 3)

In [61]:
min_10_data['label']=min_10_data['label'].replace(2,1)

In [62]:
x_values=min_10_data['combined_result']
y_values=min_10_data['label']

In [63]:
std_X = x_values.std()
desired_snr = 0.1
std_noise = std_X * desired_snr
noise = np.random.normal(0, std_noise, x_values.shape)

In [64]:
min_10_data['noisy_feature_column'] = x_values + noise
noisy_signal=min_10_data['noisy_feature_column']

In [65]:
event=[]
labe=[]
temp=[]
for index,value in y_values.items():
    if value==1:
        temp.append(noisy_signal[index])
        if (index+1)<len(y_values) and y_values[index+1]==0:
            event.append(temp)
            labe.append(1)
            temp=[]
    if value==0:
        temp.append(noisy_signal[index])
        if (index+1)<len(y_values) and y_values[index+1]==1:
            event.append(temp)
            labe.append(0)
            temp=[]

In [66]:
event=pad_sequences(event,padding='post',truncating='post',dtype=float,maxlen=50)
labe=np.array(labe)
labe=labe.reshape(-1,1)
event=event.reshape(event.shape[0], event.shape[1], 1)

In [67]:
# For 10 Min Data set
m_10=event.shape[0]
s0_10 = np.zeros((m_10, n_s))
c0_10 = np.zeros((m_10, n_s))

In [68]:
predictions_10 = model.predict([event, s0_10, c0_10])
predictions_10 = np.round(predictions_10)

10/10 [==============================] - 0s 4ms/step


In [69]:
# Get the confusion matrix
cm = confusion_matrix(labe, predictions_10)

# We will store the results in a dictionary for easy access later
per_class_accuracies = {}
classes=[0,1]
# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(classes):
    # True negatives are all the samples that are not our current GT class (not the current row) 
    # and were not predicted as the current class (not the current column)
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))

    # True positives are all the samples of our current GT class that were predicted as such
    true_positives = cm[idx, idx]

    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm)

In [70]:
print(per_class_accuracies)

{0: 0.8027681660899654, 1: 0.8027681660899654}


In [177]:
report = classification_report(predictions_10,labe)
print(report)

              precision    recall  f1-score   support

         0.0       0.93      0.75      0.83       179
         1.0       0.69      0.91      0.78       110

    accuracy                           0.81       289
   macro avg       0.81      0.83      0.81       289
weighted avg       0.84      0.81      0.81       289


**Conclusion**
It is clearn that training is easier if we have combination of Data and testing results are quite well they can get more better if we have less training loss
moreover the testing shows it is performing good on Combined set